In [1]:
import time
import socket
import shutil
import gc
from importlib import reload
from tqdm.auto import tqdm

from pathlib import Path

import torch
from torch.utils.data import DataLoader
from datasets import load_from_disk

import src.setup_embds_from_api
reload(src.setup_embds_from_api)
from src.setup_embds_from_api import path_to_galactica_folder, path_to_orig_data, path_to_raw_data, output_dir
from src.setup_embds_from_api import checkpoint_model
from src.setup_embds_from_api import device
from src.setup_embds_from_api import make_tensors, make_metadata

import openai

d:\Users\cdiet\miniconda3\envs\galactica\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets = load_from_disk(str(path_to_raw_data))

In [3]:
#Define dirs
current_time = time.strftime("%b%d_%H-%M-%S")
host = socket.gethostname()
_output_dir = Path(path_to_galactica_folder,output_dir,current_time + '_' + host)
tensors_file = Path(_output_dir, 'tensors.tsv')
metadata_file = Path(_output_dir, 'metadata.tsv')

if not _output_dir.exists():
    _output_dir.mkdir(parents=True)

dataset = datasets['test'].select(range(2048))

# copy dataset
path_to_dataset = Path(_output_dir, 'dataset.json')
dataset.save_to_disk(str(path_to_dataset))

#copy metadata
if make_metadata:
    int2str = {v: k for k,v in (dataset.features['labels']._str2int).items()}
    metadata = dataset.map(lambda seq: {'id': str(seq['id']) }, batched=False)
    metadata = metadata.map(lambda seq: {'_labels': int2str[seq['labels']] }, batched=False)
    metadata = metadata.map(lambda seq: {'text': seq['text'].replace('\t', ' ') }, batched=False)
    metadata = metadata.map(lambda seq: {'text': seq['text'].replace('\n', ' ') }, batched=False)
    metadata.to_csv(str(metadata_file), sep='\t', index=False)

Creating CSV from Arrow format: 100%|██████████| 3/3 [00:00<00:00, 24.41ba/s]                  


In [4]:
# Tensors
if make_tensors:

    dataset = dataset.remove_columns(['id', 'title', 'labels'])

    batch_size = 32

    num_steps = len(dataset) // batch_size
    progress_bar = tqdm(range(num_steps))

    # Cleaning
    gc.collect()
    torch.cuda.empty_cache()

    tensors_f = open(tensors_file, 'w')

    for i in range(num_steps):

        print("Processing batch {}/{}".format(i, num_steps))

        batch = dataset[i*batch_size:(i+1)*batch_size]
        batch = [str(e) for e in batch['text']]
        embeddings = openai.Embedding.create(input=batch, model=checkpoint_model)['data']

        for i in range(len(embeddings)):
            embd = [str(e) for e in embeddings[i]['embedding']]
            embd = '\t'.join(embd)
            tensors_f.write(embd + '\n')

        progress_bar.update(1)

    tensors_f.close()

  0%|          | 0/64 [00:00<?, ?it/s]

Processing batch 0/64


  2%|▏         | 1/64 [00:01<01:54,  1.82s/it]

Processing batch 1/64


  3%|▎         | 2/64 [00:02<01:20,  1.30s/it]

...

 97%|█████████▋| 62/64 [01:25<00:03,  1.56s/it]

Processing batch 62/64


 98%|█████████▊| 63/64 [01:26<00:01,  1.32s/it]

Processing batch 63/64


100%|██████████| 64/64 [01:27<00:00,  1.14s/it]